In [2]:
import pandas as pd
import re
import numpy as np
from utils import normalize_text, clean_text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

df = pd.read_csv("data_project.csv", sep='\t')

print("Colonnes disponibles :", df.columns.tolist())
print("Aperçu des 5 premières lignes :")
df.head(1)

Colonnes disponibles : ['venue', 'abstract', 'authors', 'n_citation', 'references', 'title', 'year', 'id', 'class']
Aperçu des 5 premières lignes :


,venue,abstract,authors,n_citation,references,title,year,id,class
0,international conference on human-computer int...,The purpose of this study is to develop a lear...,"['Makoto Satoh', 'Ryo Muramatsu', 'Mizue Kayam...",0,"['51c7e02e-f5ed-431a-8cf5-f761f266d4be', '69b6...",Preliminary Design of a Network Protocol Learn...,2013,00127ee2-cb05-48ce-bc49-9de556b93346,3


In [12]:
# Options disponibles : "none" (pas de normalisation), "stem" (pour le stemming) ou "lemma" (pour la lemmatization)
normalization_method = "lemma"  # Remplacez par "stem" ou "lemma" selon votre préférence

# 1. Combinaison du titre et de l'abstract
# Si l'abstract est manquant, on utilise une chaîne vide
df['text_full'] = df['title'].astype(str) + " " + df['abstract'].fillna("")

# 2. Application du nettoyage avec la méthode de normalisation choisie
df['clean_text'] = df['text_full'].apply(lambda x: clean_text(x, normalization_method=normalization_method))

# Affichage des premières lignes pour vérification
df['clean_text'].head()


0    preliminary design of a network protocol learn...
1    a methodology for the physically accurate visu...
2    comparison of garch neural network and support...
3    comparing gng3d and quadric error metric metho...
4    improved secret image sharing method by encodi...
Name: clean_text, dtype: object

In [13]:
# 3. Création du CountVectorizer pour le schéma TF
vectorizer_tf = CountVectorizer(stop_words='english', min_df=1, max_df=0.95)
X_tf = vectorizer_tf.fit_transform(df['clean_text'])

# Affichage de la forme de la matrice TF
print("Forme de la matrice TF :", X_tf.shape)


Forme de la matrice TF : (40596, 55324)


In [14]:
# 4. Création du TfidfVectorizer pour le schéma TF-IDF
vectorizer_tfidf = TfidfVectorizer(stop_words='english', min_df=1, max_df=0.95)
X_tfidf = vectorizer_tfidf.fit_transform(df['clean_text'])

# Affichage de la forme de la matrice TF-IDF
print("Forme de la matrice TF-IDF :", X_tfidf.shape)


Forme de la matrice TF-IDF : (40596, 55324)


In [15]:
# Calcul des occurrences totales des mots
word_counts = np.array(X_tf.sum(axis=0)).flatten()
word_freq = dict(zip(vectorizer_tf.get_feature_names_out(), word_counts))

# Affichage des mots les plus fréquents
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
print("Mots les plus fréquents :")
for word, freq in sorted_word_freq[:10]:  # Affiche les 10 mots les plus fréquents
    print(f"{word}: {freq}")


Mots les plus fréquents :
based: 16170
model: 13702
data: 11684
paper: 11620
algorithm: 10682
method: 10397
problem: 9773
using: 9771
approach: 9472
network: 8516


In [16]:
# Fonction pour obtenir le top 10 des mots et la taille du vocabulaire
def get_top_words_and_vocab_size(vectorizer, X):
    word_counts = np.array(X.sum(axis=0)).flatten()
    word_freq = dict(zip(vectorizer.get_feature_names_out(), word_counts))
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    top_10_words = sorted_word_freq[:10]
    vocab_size = len(word_freq)
    return top_10_words, vocab_size

# Comparaison des méthodes de normalisation
normalization_methods = ["none", "stem", "lemma"]
results = {}

for method in normalization_methods:
    # Nettoyage et normalisation du texte
    df['clean_text'] = df['text_full'].apply(lambda x: clean_text(x, normalization_method=method))

    # Création du CountVectorizer pour le schéma TF
    vectorizer_tf = CountVectorizer(stop_words='english', min_df=1, max_df=0.95)
    X_tf = vectorizer_tf.fit_transform(df['clean_text'])

    # Obtention du top 10 des mots et de la taille du vocabulaire
    top_10_words, vocab_size = get_top_words_and_vocab_size(vectorizer_tf, X_tf)

    # Stockage des résultats
    results[method] = {
        "top_10_words": top_10_words,
        "vocab_size": vocab_size
    }

# Affichage des résultats
for method, result in results.items():
    print(f"\nMéthode de normalisation : {method}")
    print("Top 10 des mots :")
    for word, freq in result["top_10_words"]:
        print(f"{word}: {freq}")
    print(f"Taille du vocabulaire : {result['vocab_size']}")



Méthode de normalisation : none
Top 10 des mots :
based: 16170
data: 11684
paper: 11457
using: 9771
model: 9636
approach: 7588
information: 7554
systems: 7431
algorithm: 7105
method: 6907
Taille du vocabulaire : 59716

Méthode de normalisation : stem
Top 10 des mots :
thi: 24270
use: 21694
base: 17269
model: 16690
data: 11684
paper: 11620
algorithm: 10909
method: 10447
problem: 9799
propos: 9607
Taille du vocabulaire : 43293

Méthode de normalisation : lemma
Top 10 des mots :
based: 16170
model: 13702
data: 11684
paper: 11620
algorithm: 10682
method: 10397
problem: 9773
using: 9771
approach: 9472
network: 8516
Taille du vocabulaire : 55324


# Analyse des résultats de la représentation textuelle

## 1. Taille du vocabulaire

- **Taille du vocabulaire (TF et TF-IDF)** :
  Les deux approches ont généré un vocabulaire de **23360 termes**.
  Cela indique que les paramètres de filtrage (min_df=2 et max_df=0.95) et le prétraitement (nettoyage, lemmatisation) ont été appliqués de manière cohérente pour les deux méthodes.

## 2. Fréquence des termes (TF)

- **Top 10 des mots les plus fréquents (TF)** :
  - based: 16170
  - model: 13702
  - data: 11684
  - paper: 11620
  - algorithm: 10682
  - method: 10397
  - problem: 9773
  - using: 9771
  - approach: 9472
  - network: 8516

- **Interprétation** :
  Ces termes reflètent typiquement un corpus scientifique en informatique, où les mots liés aux méthodes, aux modèles et aux données sont naturellement fréquents. La présence de termes comme *algorithm*, *model* ou *network* confirme la cohérence avec le domaine traité.

## 3. Informations sur les matrices TF et TF-IDF

- **Dimensions** :
  Les deux matrices ont la même taille : **40596 documents x 23360 termes**.

- **Densité** :
  La densité de 0.0012 montre que la matrice est très creuse, ce qui est attendu pour un grand corpus textuel où la majorité des termes ne sont présents que dans quelques documents.

- **Comparaison** :
  Bien que la structure (dimensions et densité) soit identique pour la matrice TF et celle TF-IDF, les valeurs numériques diffèrent.
  La pondération TF-IDF réduit l'importance des termes trop fréquents (moins discriminants) tout en mettant en avant les termes qui, bien que moins fréquents, apportent une information plus spécifique sur le contenu des documents.

## 4. Exemple de valeurs pour le document 0

- **Observations** :
  Pour le document 0, quelques termes et leurs valeurs associées sont :

  | Terme    | TF   | TF-IDF  |
  |----------|------|---------|
  | tool     | 4.0  | 0.3367  |
  | network  | 3.0  | 0.2067  |
  | learning | 3.0  | 0.2322  |
  | student  | 3.0  | 0.3420  |
  | design   | 2.0  | 0.1533  |
  | protocol | 2.0  | 0.2026  |
  | high     | 2.0  | 0.1648  |
  | school   | 2.0  | 0.2838  |
  | study    | 2.0  | 0.1422  |
  | develop  | 2.0  | 0.2034  |

- **Analyse** :
  - **TF (Term Frequency)** : Indique le nombre d'occurrences du terme dans le document.
  - **TF-IDF** : Intègre la fréquence du terme dans le document et son inverse de fréquence dans l'ensemble du corpus.
    - Par exemple, le terme *student* apparaît 3 fois avec un TF-IDF de 0.3420, ce qui suggère qu'il est relativement discriminant dans ce document comparé à d'autres termes plus courants dans le corpus.

## 5. Conclusion

- Le prétraitement (nettoyage, lemmatisation) a permis d’uniformiser les termes, réduisant ainsi la variance du vocabulaire tout en conservant des informations pertinentes pour l'analyse.
- L'analyse des fréquences confirme que le corpus est en adéquation avec un domaine scientifique, avec des termes spécifiques au domaine de l'informatique.
- La comparaison entre les représentations TF et TF-IDF offre deux perspectives :
  - **TF** donne une mesure brute des occurrences,
  - **TF-IDF** valorise les termes discriminants et atténue l'effet des termes très fréquents mais moins informatifs.


# Comparaison des Méthodes de Normalisation

| **Méthode de Normalisation** | **Top 10 des mots**                                                                 | **Taille du vocabulaire** |
|------------------------------|------------------------------------------------------------------------------------|---------------------------|
| Aucune                       | 1. based: 16170<br>2. data: 11684<br>3. paper: 11457<br>4. using: 9771<br>5. model: 9636<br>6. approach: 7588<br>7. information: 7554<br>8. systems: 7431<br>9. algorithm: 7105<br>10. method: 6907 | 59716                     |
| Stemming                     | 1. thi: 24270<br>2. use: 21694<br>3. base: 17269<br>4. model: 16690<br>5. data: 11684<br>6. paper: 11620<br>7. algorithm: 10909<br>8. method: 10447<br>9. problem: 9799<br>10. propos: 9607 | 43293                     |
| Lemmatisation                | 1. based: 16170<br>2. model: 13702<br>3. data: 11684<br>4. paper: 11620<br>5. algorithm: 10682<br>6. method: 10397<br>7. problem: 9773<br>8. using: 9771<br>9. approach: 9472<br>10. network: 8516 | 55324                     |

## Exercice 3


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Fonction pour traiter une requête sous forme de chaîne et la transformer en vecteur TF-IDF
def process_query(query_string, vectorizer):
    # Transformation de la requête en vecteur TF-IDF
    query_vector = vectorizer.transform([query_string])
    return query_vector

# Exemple de requête sous forme de chaîne
query_string = "Is the Macintosh Dead Meat"

# Transformation de la requête en vecteur TF-IDF
query_vector = process_query(query_string, vectorizer_tfidf)

# Calcul de la similarité cosinus entre le vecteur-requête et les vecteurs documents
cosine_similarities = cosine_similarity(query_vector, X_tfidf).flatten()

# Filtrer les documents ayant un score de similarité supérieur à 0
filtered_indices = [i for i in range(len(cosine_similarities)) if cosine_similarities[i] > 0]

# Trier les indices filtrés par ordre de similarité décroissante
sorted_filtered_indices = sorted(filtered_indices, key=lambda i: cosine_similarities[i], reverse=True)

# Récupération des documents les plus pertinents
top_n = 5  # Nombre de documents à afficher
top_documents = df.iloc[sorted_filtered_indices[:top_n]].copy()

# Ajout des scores de similarité au DataFrame
top_documents['similarity_score'] = [cosine_similarities[i] for i in sorted_filtered_indices[:top_n]]

# Affichage des documents pertinents dans un tableau DataFrame avec les scores de similarité
if not top_documents.empty:
    print("\nDocuments les plus pertinents :")
    display(top_documents[['title', 'abstract', 'similarity_score']])
else:
    print("\nAucun document pertinent trouvé.")



Documents les plus pertinents :


,title,abstract,similarity_score
30215,Is the Macintosh Dead Meat,NaN,1.000000
23117,Dead Block Placement Avoidance in L1 Data Caches.,NaN,0.224900
40066,Dead Reckoning for Mobile Robots Using Two Opt...,NaN,0.222685
40073,ESTIMATION AND COMPENSATION OF DEAD-ZONE INHER...,NaN,0.215279
24061,DOUBLE PULSE TRANSMISSION - DEAD ZONE DECREASI...,NaN,0.194699


In [ ]:
# Pour compérer tf et tf x idf

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Fonction pour traiter une requête sous forme de chaîne et la transformer en vecteur
def process_query(query_string, vectorizer):
    # Transformation de la requête en vecteur
    query_vector = vectorizer.transform([query_string])
    return query_vector

# Exemple de requête sous forme de chaîne
query_string = "machine learning model"

# Utilisation de CountVectorizer pour le schéma TF
vectorizer_tf = CountVectorizer(stop_words='english', min_df=1, max_df=0.95)
X_tf = vectorizer_tf.fit_transform(df['clean_text'])

# Utilisation de TfidfVectorizer pour le schéma TF-IDF
vectorizer_tfidf = TfidfVectorizer(stop_words='english', min_df=1, max_df=0.95)
X_tfidf = vectorizer_tfidf.fit_transform(df['clean_text'])

# Transformation de la requête en vecteurs TF et TF-IDF
query_vector_tf = process_query(query_string, vectorizer_tf)
query_vector_tfidf = process_query(query_string, vectorizer_tfidf)

# Calcul de la similarité cosinus pour TF et TF-IDF
cosine_similarities_tf = cosine_similarity(query_vector_tf, X_tf).flatten()
cosine_similarities_tfidf = cosine_similarity(query_vector_tfidf, X_tfidf).flatten()

# Filtrer et trier les documents pour TF
filtered_indices_tf = [i for i in range(len(cosine_similarities_tf)) if cosine_similarities_tf[i] > 0]
sorted_filtered_indices_tf = sorted(filtered_indices_tf, key=lambda i: cosine_similarities_tf[i], reverse=True)

# Filtrer et trier les documents pour TF-IDF
filtered_indices_tfidf = [i for i in range(len(cosine_similarities_tfidf)) if cosine_similarities_tfidf[i] > 0]
sorted_filtered_indices_tfidf = sorted(filtered_indices_tfidf, key=lambda i: cosine_similarities_tfidf[i], reverse=True)

# Récupération des documents les plus pertinents
top_n = 5  # Nombre de documents à afficher
top_documents_tf = df.iloc[sorted_filtered_indices_tf[:top_n]].copy()
top_documents_tfidf = df.iloc[sorted_filtered_indices_tfidf[:top_n]].copy()

# Ajout des scores de similarité aux DataFrames
top_documents_tf['similarity_score'] = [cosine_similarities_tf[i] for i in sorted_filtered_indices_tf[:top_n]]
top_documents_tfidf['similarity_score'] = [cosine_similarities_tfidf[i] for i in sorted_filtered_indices_tfidf[:top_n]]

# Affichage des résultats
print("\nComparaison des schémas de pondération pour la requête :", query_string)

print("\nDocuments les plus pertinents (schéma TF) :")
if not top_documents_tf.empty:
    display(top_documents_tf[['title', 'abstract', 'similarity_score']])
else:
    print("Aucun document pertinent trouvé.")

print("\nDocuments les plus pertinents (schéma TF-IDF) :")
if not top_documents_tfidf.empty:
    display(top_documents_tfidf[['title', 'abstract', 'similarity_score']])
else:
    print("Aucun document pertinent trouvé.")


In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def search_documents(query, df, normalization_method="none",
                     weighting_scheme="tf", vocab_size=None,
                     include_stopwords=False, similarity_measure="cosine",
                     top_n=10, verbose=True):
    """
    Réalise une recherche sur le corpus à partir d'une requête textuelle,
    avec différentes options de configuration.

    Paramètres :
      - query : chaîne de caractères saisie par l'utilisateur.
      - df : le DataFrame contenant les documents avec une colonne 'clean_text'.
      - normalization_method : méthode de normalisation à utiliser ("none", "stem" ou "lemma").
      - weighting_scheme : schéma de pondération ("tf" ou "tfidf").
      - vocab_size : taille du vocabulaire à utiliser (None = tout le vocabulaire).
      - include_stopwords : inclure ou non les mots-outils.
      - similarity_measure : mesure de similarité à utiliser ("cosine" ou "euclidean").
      - top_n : nombre de résultats à afficher.
      - verbose : si True, affiche les détails de configuration et les résultats.

    Retourne :
      - Une liste des indices des documents les plus similaires
      - Une liste des scores de similarité correspondants
    """
    if verbose:
        print(f"\n=== CONFIGURATION DU MOTEUR DE RECHERCHE ===")
        print(f"- Méthode de normalisation: {normalization_method}")
        print(f"- Schéma de pondération: {weighting_scheme}")
        print(f"- Taille du vocabulaire: {'complète' if vocab_size is None else vocab_size}")
        print(f"- Mots-outils: {'inclus' if include_stopwords else 'exclus'}")
        print(f"- Mesure de similarité: {similarity_measure}")

    # Appliquer le même prétraitement sur la requête
    query_clean = clean_text(query, normalization_method=normalization_method)

    # Configuration du vectorizer
    stop_words = None if include_stopwords else 'english'

    # Choix du type de vectorizer en fonction du schéma de pondération
    if weighting_scheme == "tf":
        vectorizer = CountVectorizer(stop_words=stop_words, min_df=2, max_df=0.95)
    else:  # "tfidf"
        vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2, max_df=0.95)

    # Entraînement du vectorizer sur les documents
    X = vectorizer.fit_transform(df['clean_text'])

    # Si une taille de vocabulaire est spécifiée, on limite le vocabulaire
    if vocab_size is not None and vocab_size < len(vectorizer.vocabulary_):
        # Calcul des occurrences totales de chaque mot dans le corpus
        word_counts = np.asarray(X.sum(axis=0)).flatten()
        vocab_counts = {word: word_counts[idx] for word, idx in vectorizer.vocabulary_.items()}

        # Sélection des N mots les plus fréquents
        sorted_vocab = sorted(vocab_counts.items(), key=lambda x: x[1], reverse=True)
        limited_vocab = [word for word, _ in sorted_vocab[:vocab_size]]

        # Recréation du vectorizer avec le vocabulaire limité
        if weighting_scheme == "tf":
            vectorizer = CountVectorizer(vocabulary=limited_vocab)
        else:  # "tfidf"
            vectorizer = TfidfVectorizer(vocabulary=limited_vocab)

        # Réentraînement avec le vocabulaire limité
        X = vectorizer.fit_transform(df['clean_text'])

        if verbose:
            print(f"- Vocabulaire limité aux {vocab_size} mots les plus fréquents")

    # Transformation de la requête en vecteur
    query_vector = vectorizer.transform([query_clean])

    # Calcul de la similarité selon la mesure choisie
    if similarity_measure == "cosine":
        similarities = cosine_similarity(query_vector, X).flatten()
    else:  # "euclidean"
        # Pour la distance euclidienne, on inverse les valeurs car une distance plus petite
        # signifie une plus grande similarité
        distances = euclidean_distances(query_vector, X).flatten()
        similarities = 1 / (1 + distances)  # Transformation en similarité

    # Trier les indices des documents par ordre décroissant de similarité
    sorted_indices = similarities.argsort()[::-1]
    sorted_scores = similarities[sorted_indices]

    # Affichage des résultats si verbose est activé
    if verbose:
        print(f"\nTop {top_n} documents pour la requête : '{query}'\n")
        for idx, score in zip(sorted_indices[:top_n], sorted_scores[:top_n]):
            title = df.loc[idx, 'title']
            abstract = df.loc[idx, 'abstract'] if pd.notnull(df.loc[idx, 'abstract']) else "Abstract manquant"
            print(f"Score: {score:.4f}")
            print(f"Title: {title}")
            print(f"Abstract: {abstract}")
            print("-" * 80)

    return sorted_indices[:top_n], sorted_scores[:top_n]

# Fonction pour comparer différentes configurations du moteur de recherche
def compare_search_configurations(query, df, configs, top_n=5):
    """
    Compare les résultats de recherche avec différentes configurations.

    Paramètres:
      - query: la requête à exécuter
      - df: le DataFrame contenant les documents
      - configs: liste de dictionnaires de paramètres pour search_documents
      - top_n: nombre de résultats à afficher pour chaque configuration
    """
    all_results = []

    print(f"Comparaison des résultats pour la requête: '{query}'")

    for i, config in enumerate(configs):
        print(f"\nConfiguration #{i+1}:")
        for param, value in config.items():
            if param != 'df':  # Ne pas afficher le DataFrame
                print(f"- {param}: {value}")

        # Exécuter la recherche avec cette configuration
        indices, scores = search_documents(query, df, **{**config, 'verbose': False, 'top_n': top_n})

        # Afficher les résultats
        print(f"\nTop {min(top_n, len(indices))} résultats:")
        for j, (idx, score) in enumerate(zip(indices, scores)):
            print(f"{j+1}. Score: {score:.4f}, Titre: {df.loc[idx, 'title']}")

        all_results.append({
            'config': config,
            'indices': indices,
            'scores': scores
        })

    # Analyse de la diversité des résultats
    if len(configs) > 1:
        print("\nAnalyse de la diversité des résultats:")
        for i in range(len(configs)):
            for j in range(i+1, len(configs)):
                # Calcul du taux de recouvrement entre les résultats des configurations i et j
                set_i = set(all_results[i]['indices'])
                set_j = set(all_results[j]['indices'])
                overlap = len(set_i.intersection(set_j))
                overlap_rate = overlap / min(len(set_i), len(set_j))

                print(f"Recouvrement entre config #{i+1} et config #{j+1}: {overlap}/{min(len(set_i), len(set_j))} documents ({overlap_rate:.2%})")

    return all_results

# Exemple d'utilisation pour comparer différentes configurations
def run_search_comparison(query, df):
    """
    Exemple d'utilisation montrant comment comparer différentes configurations
    du moteur de recherche pour une requête donnée.
    """
    # Définir différentes configurations à tester
    configs = [
        {
            # Configuration de base: TF, sans mots-outils, similarité cosinus
            'normalization_method': 'none',
            'weighting_scheme': 'tf',
            'include_stopwords': False,
            'similarity_measure': 'cosine',
            'vocab_size': None
        },
        {
            # Configuration avec TF-IDF
            'normalization_method': 'none',
            'weighting_scheme': 'tfidf',
            'include_stopwords': False,
            'similarity_measure': 'cosine',
            'vocab_size': None
        },
        {
            # Configuration avec vocabulaire limité
            'normalization_method': 'none',
            'weighting_scheme': 'tf',
            'include_stopwords': False,
            'similarity_measure': 'cosine',
            'vocab_size': 500
        },
        {
            # Configuration avec mots-outils
            'normalization_method': 'none',
            'weighting_scheme': 'tf',
            'include_stopwords': True,
            'similarity_measure': 'cosine',
            'vocab_size': None
        },
        {
            # Configuration avec distance euclidienne
            'normalization_method': 'none',
            'weighting_scheme': 'tf',
            'include_stopwords': False,
            'similarity_measure': 'euclidean',
            'vocab_size': None
        }
    ]

    # Comparer les résultats avec ces différentes configurations
    compare_search_configurations(query, df, configs)

# Exemple d'utilisation avec une requête spécifique
# run_search_comparison("machine learning", df)


Comparaison des schémas de pondération pour la requête : machine learning model

Documents les plus pertinents (schéma TF) :


,title,abstract,similarity_score
35626,Using Machine Learning Methods to Improve Qual...,NaN,0.666667
36360,Extending extreme learning machine with combin...,We consider the Extreme Learning Machine model...,0.604145
13535,Machine learning and agents,The paper reviews current research results int...,0.601793
25141,Learning Hebrew Roots: Machine Learning with L...,NaN,0.577350
26165,Machine Learning and Relevance Feedback.,NaN,0.577350



Documents les plus pertinents (schéma TF-IDF) :


,title,abstract,similarity_score
13535,Machine learning and agents,The paper reviews current research results int...,0.603422
38803,ASV monitor: creating comparability of machine...,In this demonstration paper we present an appl...,0.568691
35626,Using Machine Learning Methods to Improve Qual...,NaN,0.566570
33050,Sparse Representation for Machine Learning,NaN,0.552298
32272,Visual Data Mining and Machine Learning.,NaN,0.537334


In [20]:
query = "beach aristocrat"  # Exemple de requête
run_search_comparison(query, df)

Comparaison des résultats pour la requête: 'beach aristocrat'

Configuration #1:
- normalization_method: none
- weighting_scheme: tf
- include_stopwords: False
- similarity_measure: cosine
- vocab_size: None

Top 5 résultats:
1. Score: 0.0729, Titre: Development of an Information Portal for the Lake Winnipeg Basin Initiative
2. Score: 0.0702, Titre: An adaptive pitch control strategy for a doubly fed wind generation system
3. Score: 0.0000, Titre: An object-oriented denotational semantics of a small programming language.
4. Score: 0.0000, Titre: Extending the Object-Process Methodology to Handle Real-Time Systems.
5. Score: 0.0000, Titre: Query Execution for RDF Data on Row and Column Store

Configuration #2:
- normalization_method: none
- weighting_scheme: tfidf
- include_stopwords: False
- similarity_measure: cosine
- vocab_size: None

Top 5 résultats:
1. Score: 0.1133, Titre: Development of an Information Portal for the Lake Winnipeg Basin Initiative
2. Score: 0.1112, Titre: An adap

# Analyse des résultats de la comparaison des configurations du moteur de recherche

La comparaison a été effectuée pour la requête **"machine learning"** en testant cinq configurations différentes. Voici un résumé et une analyse détaillée :

---

## 1. Configurations testées

- **Configuration #1 :**
  - **Paramètres :** TF, sans normalisation, sans inclusion de stop words, vocabulaire complet, similarité cosinus.
  - **Résultats :**
    Les documents affichés (scores autour de 0.7071 pour les quatre premiers et 0.6937 pour le cinquième) suggèrent une forte similarité entre la requête et les documents sélectionnés, avec une cohérence dans le classement.

- **Configuration #2 :**
  - **Paramètres :** TF-IDF, sans normalisation, sans inclusion de stop words, vocabulaire complet, similarité cosinus.
  - **Résultats :**
    Les scores sont légèrement plus faibles (allant de 0.6518 à 0.5667) et l'ordre des résultats change par rapport à la configuration #1. Cela reflète l'effet de la pondération TF-IDF qui tend à atténuer l'influence des termes très fréquents et moins discriminants.

- **Configuration #3 :**
  - **Paramètres :** TF, sans normalisation, sans inclusion de stop words, vocabulaire limité aux 500 mots les plus fréquents, similarité cosinus.
  - **Résultats :**
    On observe des scores très élevés (deux documents avec un score parfait de 1.0000) et une plus grande différenciation entre les documents. La limitation du vocabulaire semble concentrer la représentation sur un ensemble de termes très pertinents, ce qui conduit à une correspondance plus forte pour certains documents.

- **Configuration #4 :**
  - **Paramètres :** TF, sans normalisation, avec inclusion de stop words, vocabulaire complet, similarité cosinus.
  - **Résultats :**
    Les scores sont modérément faibles (entre 0.6708 et 0.5774). L'inclusion des stop words modifie légèrement le profil de la similarité et peut introduire du bruit, entraînant ainsi un classement différent de celui obtenu sans stop words.

- **Configuration #5 :**
  - **Paramètres :** TF, sans normalisation, sans inclusion de stop words, vocabulaire complet, **mesure de similarité euclidienne**.
  - **Résultats :**
    Tous les documents affichés ont le même score (0.4142) et les titres semblent moins pertinents par rapport à la requête. Ce résultat illustre que, pour ce corpus et cette requête, la mesure euclidienne ne capture pas efficacement la notion de similarité que la cosinus peut exprimer.

---

## 2. Analyse des recouvrements entre configurations

Le taux de recouvrement entre les résultats de différentes configurations indique la similitude des classements obtenus :

- **Entre config #1 et #2 :** 60% des documents se recoupent
  → Indique que TF et TF-IDF, bien que différents dans la pondération, conduisent à des résultats relativement similaires.

- **Entre config #1 et #3 :** 40% de recouvrement
  → La limitation du vocabulaire modifie significativement les résultats, concentrant la recherche sur un sous-ensemble de termes.

- **Entre config #1 et #4 :** 80% de recouvrement
  → L'inclusion des stop words, dans ce cas, n'altère pas énormément le classement, même si quelques ajustements sont observés.

- **Entre config #1 et #5 :** 20% de recouvrement
  → La différence de mesure de similarité (euclidienne vs cosinus) a un impact très marqué sur les documents retenus.

- **Autres recouvrements faibles (0–20%) avec la config #5**
  → Confirment que la mesure euclidienne produit un classement radicalement différent, probablement moins pertinent pour ce type de données textuelles.

---

## 3. Conclusion

- **Choix de la pondération :**
  - **TF vs TF-IDF :** La pondération TF-IDF réduit l'effet des termes très fréquents, modifiant légèrement l'ordre des documents, mais reste globalement cohérente avec TF (60% de recouvrement).

- **Taille du vocabulaire :**
  - Limiter le vocabulaire (config #3) peut augmenter les scores pour certains documents (scores parfaits), ce qui peut être bénéfique pour une recherche focalisée, mais cela modifie aussi le classement et réduit la diversité des résultats.

- **Inclusion des stop words :**
  - Leur inclusion (config #4) affecte moins fortement le classement, même si l'on peut observer quelques différences, probablement dues à l'introduction d'un bruit modéré.

- **Mesure de similarité :**
  - La similarité cosinus semble mieux adaptée aux représentations textuelles (TF et TF-IDF) qu'une mesure basée sur la distance euclidienne, qui, dans ce cas, génère des scores très homogènes et des résultats moins pertinents.

Dans l'ensemble, cette comparaison met en évidence l'importance de choisir soigneusement les paramètres (pondération, vocabulaire, stop words, et mesure de similarité) pour optimiser la performance d'un moteur de recherche sur un corpus donné.
